In [ ]:
from transformers import pipeline
import os
import pandas as pd
import re
from collections import defaultdict
from tqdm import tqdm
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model='nicholasKluge/TeenyTinyLlama-160m-IMDB',
    trust_remote_code=True
)

In [ ]:
def read_txt_file(path):
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except Exception as e:
        print(f"Error on reading {path}: {e}")
        return ''

# Remove patterns like "word/ em os" and maintain just the word before the backslash
def clean_opinosis_text(text):
    return re.sub(r'([\wãõáéíóúâêôç]+)\/_ em os', r'\1', text)

# Remove tags like <...> at the end of each line
def clean_extractive_human_text(text):
    lines = text.splitlines()
    cleaned_lines = [re.sub(r'\s*<[^>]+>\s*$', '', line) for line in lines]
    return '\n'.join(cleaned_lines)

# Função para coletar todos os resumos de um subject
def coletar_resumos(base_dir):
    subjects = set()
    data = defaultdict(dict)

    # 1. Summaries from Opinosis and LLM
    generated_sum_dir = os.path.join(base_dir, 'resumos_gerados')
    for subject in tqdm(os.listdir(generated_sum_dir), desc='Colecting generated summaries:'):
        subject_path = os.path.join(generated_sum_dir, subject)
        if not os.path.isdir(subject_path) or subject == 'OpiSums-PT':
            continue
        subjects.add(subject)

        extractive_llm = read_txt_file(os.path.join(subject_path, 'llm', 'resumo_extrativo.txt'))
        abstractive_llm = read_txt_file(os.path.join(subject_path, 'llm', 'resumo_abstrativo.txt'))
        opinosis = read_txt_file(os.path.join(subject_path, 'opinosis', 'resumo.txt'))
        opinosis = clean_opinosis_text(opinosis)
        
        # Storing summaries
        data[subject]['LLM Extrativo'] = extractive_llm
        data[subject]['LLM Abstrativo'] = abstractive_llm
        data[subject]['Opinosis'] = opinosis

    # 2. Human summaries (Abstractives e Extractives)
    summaries_dir = os.path.join(base_dir, 'OpiSums-PT', 'Sumarios')
    for subject in tqdm(os.listdir(summaries_dir), desc='Colecting human summaries:'):
        subject_path = os.path.join(summaries_dir, subject)
        if not os.path.isdir(subject_path):
            continue
        subjects.add(subject)

        abstractives_dir = os.path.join(subject_path, 'Abstrativos')
        extrativos_dir = os.path.join(subject_path, 'Extrativos')

        abstractive_files = sorted([f for f in os.listdir(abstractives_dir) if f.endswith('.txt')])[:5]
        extractive_files = sorted([f for f in os.listdir(extrativos_dir) if f.endswith('.txt')])[:5]

        for idx, fname in enumerate(abstractive_files):
            abstractive_file = os.path.join(abstractives_dir, fname)
            data[subject][f'Humano_Abstrativo_{idx+1}'] = read_txt_file(abstractive_file)
        for idx, fname in enumerate(extractive_files):
            extrativo_path = os.path.join(extrativos_dir, fname)
            data[subject][f'Humano_Extrativo_{idx+1}'] = clean_extractive_human_text(read_txt_file(extrativo_path))
    return data, sorted(list(subjects))

# Function to run sentiment analysis on all texts of a subject
def analyze_feelings_for_subject(data_subject, sentiment_analyzer):
    results = {}
    for key, text in data_subject.items():
        if text:
            try:
                result = sentiment_analyzer(text)
                if isinstance(result, list):
                    result = result[0]
                label = result['label']
                score = result['score']
                results[key] = (label, score, text)
            except Exception as e:
                print(f"Error on analyzing feeling for {key}: {e}")
                results[key] = ('ERROR', 0.0, text)
        else:
            results[key] = ('EMPTY', 0.0, text)
    return results

def create_feeling_report(base_dir, sentiment_analyzer):
    data, subjects = coletar_resumos(base_dir)
    lines = []
    columns_fll = ['Assunto', 'LLM Extrativo', 'LLM Abstrativo', 'Opinosis']
    columns_text = ['Texto LLM Extrativo', 'Texto LLM Abstrativo', 'Texto Opinosis']
    columns_score = ['LLM Extrativo Nota', 'LLM Abstrativo Nota', 'Opinosis Nota']
    for i in range(1, 6):
        columns_fll.append(f'Humano_Abstrativo_{i}')
        columns_fll.append(f'Humano_Extrativo_{i}')
        columns_score.append(f'Humano_Abstrativo_{i} Nota')
        columns_score.append(f'Humano_Extrativo_{i} Nota')
        columns_text.append(f'Texto Humano_Abstrativo_{i}')
        columns_text.append(f'Texto Humano_Extrativo_{i}')
    columns = ['Assunto']
    for c, n in zip(columns_fll[1:], columns_score):
        columns.append(c)
        columns.append(n)
    columns += columns_text

    for subject in tqdm(subjects, desc='Processing subjects'):
        data_subject = data[subject]
        results = analyze_feelings_for_subject(data_subject, sentiment_analyzer)
        line = [subject]

        for key in columns_fll[1:]:
            if key in results:
                line.append(results[key][0])
                line.append(round(results[key][1], 3))
            else:
                line.append('N/A')
                line.append(0.0)

        for key in columns_fll[1:]:
            if key in results:
                line.append(results[key][2])
            else:
                line.append('')
        lines.append(line)
    df = pd.DataFrame(lines, columns=columns)
    return df

In [ ]:
base_dir = '/content/drive/MyDrive/opinosis_tilic'
report_df = create_feeling_report(base_dir, sentiment_analyzer)

In [ ]:
display(report_df)

,Assunto,LLM Extrativo,LLM Extrativo Nota,LLM Abstrativo,LLM Abstrativo Nota,Opinosis,Opinosis Nota,Humano_Abstrativo_1,Humano_Abstrativo_1 Nota,Humano_Extrativo_1,...,Texto Humano_Abstrativo_1,Texto Humano_Extrativo_1,Texto Humano_Abstrativo_2,Texto Humano_Extrativo_2,Texto Humano_Abstrativo_3,Texto Humano_Extrativo_3,Texto Humano_Abstrativo_4,Texto Humano_Extrativo_4,Texto Humano_Abstrativo_5,Texto Humano_Extrativo_5
0,1984,POSITIVE,0.997,POSITIVE,0.997,POSITIVE,0.883,POSITIVE,0.890,POSITIVE,...,O livro 1984 recebeu boas críticas pela maiori...,""" Um ótimo livro, além de ser um ótimo alerta ...",﻿Mais da metade dos leitores do livro “1984” a...,﻿O livro talvez desanime leitores mais afoito ...,"O livro 1984 apresenta, de forma interessante,...",Contém medo e suspense desde a primeira linha....,De forma geral a história do livro 1984 é con...,﻿A história em si é interessante.\nContém medo...,"﻿Sobre o livro 1984, escrito por George Orwell...",A história em si é interessante.\nBom livro.\n...
1,Capitaes-da-Areia,POSITIVE,0.990,POSITIVE,0.997,POSITIVE,0.947,POSITIVE,0.951,POSITIVE,...,Capitães da Areia é um livro brasileiro escrit...,A narrativa nos surpreende com o decorrer de o...,O livro Capitães da Areia considerado um bom l...,﻿Um livro muito bom que retrata a cruel realid...,"﻿O livro Capitães da Areia, de maneira envolve...","﻿Publicado em 1937, a obra parece conter os in...","Em geral, os usuários afirmam que a obra ""Capi...","Eu adorei esse livro quando li, foi um conselh...","﻿O livro Capitães da Areia, de autoria do reco...",﻿Um livro muito bom que retrata a cruel realid...
2,Crepusculo,NEGATIVE,0.996,POSITIVE,0.937,POSITIVE,0.939,NEGATIVE,0.978,NEGATIVE,...,﻿A maioria definitivamente não gostou do livro...,"﻿A história tem vários furos e, se você parar ...",﻿O livro Crepúsculo conquistou milhares de pes...,"﻿História clichê, recheada de coisas impossíve...",﻿O livro Crepúsculo recebeu comentários negati...,Abandonei o livro crepúsculo por que eu gosto ...,A grande maioria dos leitores avaliaram negati...,Decepcionante.\nTédio ...\nA história tem vári...,"﻿Apesar de haver quem goste de Crepúsculo, o l...","Intenso, romantico, inesperado, brilhante, apa..."
3,Ensaio-Sobre-a-Cegueira,POSITIVE,0.874,POSITIVE,0.998,POSITIVE,0.555,POSITIVE,0.538,POSITIVE,...,﻿Todos reconhecem que “Ensaio sobre a Cegueira...,﻿É um livro que prendeu minha atenção de o com...,"﻿Um livro extraordinário, que suscita profunda...",﻿Surreal\nadaptabilidade\nA obra mostra como p...,Este livro de Saramago leva a uma intensa refl...,"Em os livros de Saramago, voce encontra histor...",A opinião dos usuários sobre o livro “Ensaio ...,﻿O livro de Saramago é espetacular.\nEm síntes...,"﻿Em Ensaio Sobre a Cegueira, o autor Saramago ...",A maneira como Saramago conduz a história tem ...
4,Fala-Serio-Amiga,POSITIVE,0.994,POSITIVE,0.993,NEGATIVE,0.901,POSITIVE,0.984,POSITIVE,...,"Em geral o livro Fala Sério, amiga agradou as ...","Esse livro é um livro divertido, que a o mesmo...",﻿É unânime entre os leitores que o livro “Fala...,"﻿O livro, de Thalita Rebouças narra a história...","Os comentários sobre o livro “Fala sério, amig...","﻿Mais uma vez, em um enredo simples e muito di...","Os leitores de ""Fala sério, amiga!"", de Thalit...","﻿Mais uma vez, em um enredo simples e muito di...","﻿Fala sério, amiga! é um livro muito legal. Es...","Esse livro é um livro divertido, que a o mesmo..."
5,Fala-Serio-Amor,POSITIVE,0.998,POSITIVE,0.998,POSITIVE,0.983,POSITIVE,0.974,POSITIVE,...,"﻿Fala sério, amor! é um livro voltado para ado...","fala sério, amor!\neste livro ele é muito ente...","Um livro muito bom, principalmente para os mai...",Um ótimo livro pra quem quer dar boas risadas ...,"﻿Todos concordam que “Fala sério, amor!” é um ...","De essa vez nossa querida Thalita Rebouças "" a...",A grande maioria dos leitores aprovaram o livr...,Um ótimo livro pra quem quer dar boas risadas ...,"O livro ""Fala sério, amor"", de Talita Rebouças...",Esse livro retrata tí

In [ ]:
report_df.to_csv("relatorio_analise_sentimentos.csv")